# NLP (Natural Language Processing) with PEFT (Parameter Efficient Fine-Tuning) and LoRA (Low-Rank Adaptation) for Less-Toxic Summarization


**Flujo de trabajo del proyecto:**
* **Configuración:** Importar las bibliotecas necesarias y definir los parámetros del proyecto.
* **Exploración del conjunto de datos:** Descubrir el conjunto de datos DialogSum.
* **Probar la inferencia de cero disparos del modelo:** Inicialmente, probar el modelo FLAN-T5 para la inferencia de cero disparos en tareas de resumen de diálogos para establecer un rendimiento de referencia.
* **Preprocesar el diálogo y el resumen del conjunto de datos:** Preprocesar el diálogo y su resumen correspondiente del conjunto de datos para prepararlo para el entrenamiento.
* **Realizar un ajuste fino eficiente de parámetros (PEFT):** Implementar el ajuste fino eficiente de parámetros (PEFT), un enfoque de ajuste fino más eficiente que puede reducir significativamente el tiempo de entrenamiento mientras se mantiene el rendimiento.
* **Evaluación:**
  * Realizar una evaluación humana para medir el resultado del modelo en términos de legibilidad y coherencia. Esto puede implicar que los anotadores clasifiquen los resúmenes generados por calidad.
  * Utilizar las métricas ROUGE para evaluar la calidad de los resúmenes generados. ROUGE mide la superposición entre los resúmenes generados y las referencias escritas por humanos.

**Contextualización formal**

- **PPO-Clipping**
$$
L^{\text{PPO}}(\theta) = \mathbb{E}_t \left[ \min \left( r_t(\theta) \cdot A_t,\; \text{clip}(r_t(\theta),\; 1 - \epsilon,\; 1 + \epsilon) \cdot A_t \right) \right]
$$

PPO utiliza clipping del ratio de probabilidd para evitar actualizaciones grandes. No se mide directamente en cada época, no se puede trackear explicitamente. Se utiliza para mantener estabilidad básica del PPO.

- **KL-divergence **
$$
\text{KL}(\pi_{\theta} \;\|\; \pi_{\text{ref}}) = \mathbb{E}{s,a} \left[ \log \left( \frac{\pi{\theta}(a|s)}{\pi_{\text{ref}}(a|s)} \right) \right]
$$

Una medida directa de cuánto se ha alejado la politica actual de la politcia de referencia. Podemos incluir el KL como parte explicita de loss

- **PPO + penalización KL**

$$
L^{\text{Total}}(\theta) = L^{\text{PPO}}(\theta) - \beta \cdot \text{KL}(\pi_{\theta} \;\|\; \pi_{\text{ref}})
$$

$$\beta$$ es el coficiente de penalización KL (puede ajustarse dinámicamente)

Esto está en Huggin Face tlr



In [ ]:
%pip install datasets
%pip install trl==0.11.3
%pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━ 287.1/664.8 MB 7.7 MB/s eta 0:00:50

In [ ]:
from datasets import  load_dataset, Dataset
from transformers import pipeline, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments, GenerationConfig,Trainer
from trl import PPOTrainer, PPOConfig, AutoModelForSeq2SeqLMWithValueHead #para predecir mi V(st)
from trl import create_reference_model
from trl.core import LengthSampler

import torch
import time
import evaluate
import pandas as pd
import numpy as np


#tqdm library makes the loops show a smart progress meter
from tqdm import tqdm
tqdm.pandas()

**Datos**

DialogSum es un conjunto de datos de resumen de diálogos a gran escala, que consta de 13 460 diálogos (más 100 datos de reserva para la generación de temas) con resúmenes y temas correspondientes etiquetados manualmente.

[Dialogsum](https://huggingface.co/datasets/knkarthick/dialogsum?row=0)

## <b>1 <span style='color:#78D118'>|</span> Introducción</b>

Este proyecto explora las capacidades de los modelos de lenguaje grandes (LLM), haciendo especial hincapié en el uso del ajuste fino con eficiencia de parámetros (PEFT) para crear resúmenes de diálogos con toxicidad reducida. Ajustaremos un modelo FLAN-T5 para generar contenido menos tóxico utilizando el modelo de recompensa por discurso de odio de Meta AI. Este modelo de recompensa es un clasificador binario que predice si un texto determinado es “no odioso” o “odio”. Utilizaremos el Proximal Policy Optimization (PPO) para ajustar el modelo y reducir su toxicidad.

Nuestro objetivo principal es mejorar la calidad de los resúmenes de diálogos y, al mismo tiempo, minimizar la toxicidad. Para lograrlo, aplicamos el Proximal Policy Optimization (PPO) para el ajuste fino, con el objetivo de mitigar la salida tóxica del modelo. Además, mostraremos las ventajas del ajuste fino con eficiencia de parámetros (PEFT), demostrando que sus beneficios superan cualquier posible compensación menor en el rendimiento.



**NOTA**: Este es un ejemplo y no utilizamos la totalidad de los datos utilizados.

## <b>2 <span style='color:#78D118'>|</span> Descarga de datos</b>

Aquí, utilizaremos el modelo T5 como base entrenada previamente y utilizaremos el tokenizador correspondiente. Puede utilizar un modelo entrenado previamente diferente (y el tokenizador correspondiente) cambiando el nombre del modelo a continuación por un modelo diferente en Hugging Face Hub, o utilizar un modelo personalizado/entrenar un tokenizador desde cero en su propio conjunto de datos. Tenga en cuenta que necesitará muchos más datos y cálculos para entrenar un buen modelo desde cero.

T5 está disponible en diferentes size, including: T5 Small, T5 Base, T5 Large, T5 3B, T5 11B

In [ ]:
model_name = "google/flan-t5-base"
huggingface_dataset_name = "knkarthick/dialogsum"

# Load the dataset
dataset_original = load_dataset(huggingface_dataset_name)

# Check the dataset
print(dataset_original)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.65k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/442k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})


## <b>3 <span style='color:#78D118'>|</span> PEFT parameter visualization</b>

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"\ntrainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

## <b>4<span style='color:#78D118'>|</span> Tokenizando la información</b>


El siguiente paso implica el preprocesamiento del conjunto de datos. Seleccionaremos un subconjunto de los datos, filtraremos los diálogos a una longitud específica para garantizar la legibilidad manteniendo al mismo tiempo un contenido significativo y luego integraremos cada diálogo con una instrucción antes de convertir en tokens las indicaciones. Los identificadores de token resultantes se almacenarán en el campo `input_ids`, mientras que las indicaciones decodificadas se guardarán en el campo `query`.

Para agilizar este proceso, es recomendable crear una función llamada `build_dataset`. Esta función se puede definir de la siguiente manera:

In [ ]:
def build_dataset(model_name,
                  dataset_name,
                  input_min_text_length,
                  input_max_text_length):

    """
    Preprocess the dataset and split it into train and test parts.

    Parameters:
    - model_name (str): Tokenizer model name.
    - dataset_name (str): Name of the dataset to load.
    - input_min_text_length (int): Minimum length of the dialogues.
    - input_max_text_length (int): Maximum length of the dialogues.

    Returns:
    - dataset_splits (datasets.dataset_dict.DatasetDict): Preprocessed dataset containing train and test parts.
    """

    # load dataset (only "train" part will be enough for this lab).
    dataset = load_dataset(dataset_name, split="train")

    # Filter the dialogues of length between input_min_text_length and input_max_text_length characters.
    dataset = dataset.filter(lambda x: len(x["dialogue"]) > input_min_text_length and len(x["dialogue"]) <= input_max_text_length, batched=False)

    # Prepare tokenizer. Setting device_map="auto" allows to switch between GPU and CPU automatically.
    tokenizer = AutoTokenizer.from_pretrained(model_name) #, device_map="auto"

    def tokenize(sample):

        # Wrap each dialogue with the instruction.
        prompt = f"""
Summarize the following conversation.

{sample["dialogue"]}

Summary:
"""
        sample["input_ids"] = tokenizer.encode(prompt)

        # This must be called "query", which is a requirement of our PPO library.
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    # Tokenize each dialogue.
    dataset = dataset.map(tokenize, batched=False)
    dataset.set_format(type="torch")

    # Split the dataset into train and test parts.
    dataset_splits = dataset.train_test_split(test_size=0.2, shuffle=False, seed=42)

    return dataset_splits

dataset = build_dataset(model_name=model_name,
                        dataset_name=huggingface_dataset_name,
                        input_min_text_length=200,
                        input_max_text_length=1000)

print(dataset)

Filter:   0%|          | 0/12460 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Map:   0%|          | 0/10022 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'query'],
        num_rows: 8017
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'query'],
        num_rows: 2005
    })
})


# <b>5 <span style='color:#78D118'>|</span>  Modelo FLAN-T5 optimizado con instrucciones de resumen</b>

## <b>5.1 <span style='color:#78D118'>|</span>  Mejora del modelo FLAN-T5 optimizado con un adaptador de resumen</b>.

Estamos mejorando el modelo FLAN-T5 original agregando un adaptador de resumen. Este adaptador está diseñado para mejorar el rendimiento del modelo en tareas de resumen. Haremos un fine-tunning con el dataset mencionado

Comenzamos configurando el adaptador utilizando los siguientes parámetros:
- `r`: Rank, which is set to 32.
- `lora_alpha`: LORA alpha value, set to 32.
- `target_modules`: We specify the target modules as ["q", "v"].
- `lora_dropout`: Dropout rate for LORA, set to 0.05.
- `bias`: We use "none" as the bias configuration.
- `task_type`: The task type is set to SEQ_2_SEQ_LM, which is suitable for FLAN-T5.

A continuación, cargamos el modelo FLAN-T5 previamente entrenado y creamos una instancia de AutoModelForSeq2SeqLM con el nombre de modelo y el tipo de datos especificados (torch_dtype).

También creamos un PeftModel incorporando el modelo cargado previamente.
Además, proporcionamos la configuración de LORA, el tipo de datos torch, el mapeo del dispositivo y especificamos que el modelo se puede entrenar.

Ejemplo de LoRA:
Supón que
- La matrix Query tiene forma
𝑊_q
∈
R^(512×512)

Entonces:

- 𝐴
∈
𝑅
512
×
32



- 𝐵
∈
𝑅
32
×
512



➡️ En lugar de entrenar 262,144 parámetros (
512
×
512
), entrenas solo 32,768 (
512
×
32
+
32
×
512
).

Aqui solo estamos cargando los pesos LoRA, no todo el modelo base, por eso hace mas eficiente el entrenamiento porque solo nos centramos en ***q*** y ***v*** con matrices de pesos ***A*** y ***B*** reducidas

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType
from peft import PeftModel, PeftConfig

lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name,
                                              torch_dtype=torch.bfloat16)
## Fine-tunning
peft_model = PeftModel.from_pretrained(model,
                                       'z7ye/peft-dialogue-summary-checkpoint',
                                       lora_config=lora_config,
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=True) #device_map="auto",

print(f'PEFT model parameters to be updated:\n{print_number_of_trainable_model_parameters(peft_model)}\n')

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/334 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/14.2M [00:00<?, ?B/s]

PEFT model parameters to be updated:

trainable model parameters: 3538944
all model parameters: 251116800
percentage of trainable model parameters: 1.41%



## <b>5.2 <span style='color:#78D118'>|</span> Mejorar el resumen de LLM con aprendizaje de refuerzo con POO</b>

Ahora, estamos en el proceso de preparación para el ajuste fino del modelo de lenguaje (LLM) mediante aprendizaje por refuerzo (RL). Aunque se trata de una explicación más detallada del RL, nuestro enfoque actual está en la configuración del modelo de optimización de política proximal (PPO).

Este modelo PPO recibirá el modelo PEFT ajustado por instrucción como entrada y se utilizará para optimizar la política de RL de acuerdo con el modelo de recompensa.

Recordar que necesitamos un "modelo" para V(St) (que estima el valor esperado), aqui lo estamos agregando al modelo PEFT.

In [ ]:
ppo_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(peft_model,
                                                               torch_dtype=torch.bfloat16,
                                                               is_trainable=True)

print(f'PPO model parameters to be updated (ValueHead + NNN params):\n{print_number_of_trainable_model_parameters(ppo_model)}\n')
print(ppo_model.v_head)

PPO model parameters to be updated (ValueHead + NNN params):

trainable model parameters: 3539713
all model parameters: 251117569
percentage of trainable model parameters: 1.41%

ValueHead(
  (dropout): Dropout(p=0.1, inplace=False)
  (summary): Linear(in_features=768, out_features=1, bias=True)
  (flatten): Flatten(start_dim=1, end_dim=-1)
)


Creamos una copia congelada del modelo actual (con LoRA y la cabeza de valor), que se usará como **politica de referencia** durante el entrenamiento con PPO. Recordar Clipping por ejemplo o bien, KL-divergence

Nota: Dado que está congelado, a este modelo no se le actualizaran ningún parámetro

In [ ]:
ref_model = create_reference_model(ppo_model)

print(f'Reference model parameters to be updated:\n{print_number_of_trainable_model_parameters(ref_model)}\n')

Reference model parameters to be updated:

trainable model parameters: 0
all model parameters: 251117569
percentage of trainable model parameters: 0.00%



# <b>6<span style='color:#78D118'>|</span> Construcción de un modelo de recompensa para el aprendizaje por refuerzo</b>

El aprendizaje por refuerzo (RL) es una rama fundamental del aprendizaje automático en la que los agentes toman decisiones dentro de un entorno para maximizar sus recompensas acumuladas. El comportamiento de estos agentes está regido por una política de toma de decisiones y el objetivo fundamental del RL es que el agente adquiera una política óptima o casi óptima que maximice la función de recompensa.

Anteriormente, la política original se basaba en el modelo PEFT de instrucciones, esencialmente, el modelo de lenguaje (LLM) antes de someterse a la desintoxicación. Si bien un enfoque implicaba solicitar a los etiquetadores humanos que proporcionaran comentarios sobre la toxicidad de los resultados del modelo, este proceso puede volverse prohibitivamente costoso cuando se aplica durante toda la fase de ajuste fino. Una solución pragmática para evitar este gasto es implementar un modelo de recompensa que aliente al agente a producir resúmenes de diálogo desintoxicados.

Un enfoque sensato en este caso es realizar un análisis de sentimientos sobre los resultados del modelo, clasificándolos en dos categorías: "nothate" y "hate". Se asignan recompensas más altas cuando la probabilidad de clasificar un resultado como "nothate" es mayor.

En este contexto, emplearemos el [Meta AI's RoBERTa-based hate speech model](https://huggingface.co/facebook/roberta-hate-speech-dynabench-r4-target) como nuestro modelo de recompensa. Este modelo genera **logits** y, posteriormente, predice probabilidades para dos clases: "nothate" y "hate". Las recompensas positivas se derivan de los logits asociados con la clase "nothate". El modelo se someterá a un ajuste adicional mediante la optimización de políticas proximales (PPO) con estos valores de recompensa.

## <b>6.1<span style='color:#78D118'>|</span> Cargue el modelo de discurso de odio basado en RoBERTa de Meta AI</b>

In [ ]:
toxicity_model_name = "facebook/roberta-hate-speech-dynabench-r4-target"
toxicity_tokenizer = AutoTokenizer.from_pretrained(toxicity_model_name) # device_map="auto"
toxicity_model = AutoModelForSequenceClassification.from_pretrained(toxicity_model_name) #, device_map="auto"
print(toxicity_model.config.id2label)

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/816 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

{0: 'nothate', 1: 'hate'}


Tomemos un texto no tóxico y vamos a convertirlo en un token y pasarlo al modelo. Imprimiremos los logits de salida, las probabilidades y la recompensa correspondiente que se utilizará para el ajuste fino.

In [ ]:
non_toxic_text = "#Person 1# tells Tommy that he didn't like the movie."

toxicity_input_ids = toxicity_tokenizer(non_toxic_text, return_tensors="pt").input_ids

logits = toxicity_model(input_ids=toxicity_input_ids).logits
print(f'logits [not hate, hate]: {logits.tolist()[0]}')

# Print the probabilities for [not hate, hate]
probabilities = logits.softmax(dim=-1).tolist()[0]
print(f'probabilities [not hate, hate]: {probabilities}')

# get the logits for "not hate" - this is the reward!
not_hate_index = 0
nothate_reward = (logits[:, not_hate_index]).tolist()
print(f'reward (high): {nothate_reward}')

logits [not hate, hate]: [3.11409854888916, -2.4896156787872314]
probabilities [not hate, hate]: [0.9963293671607971, 0.00367063214071095]
reward (high): [3.11409854888916]


Mostremos otro comentario. Este tendrá una recompensa baja porque es más tóxico.

In [ ]:
toxic_text = "#Person 1# tells Tommy that the movie was terrible, dumb and stupid."

toxicity_input_ids = toxicity_tokenizer(toxic_text, return_tensors="pt").input_ids

logits = toxicity_model(toxicity_input_ids).logits
print(f'logits [not hate, hate]: {logits.tolist()[0]}')

# Print the probabilities for [not hate, hate]
probabilities = logits.softmax(dim=-1).tolist()[0]
print(f'probabilities [not hate, hate]: {probabilities}')

# Get the logits for "not hate" - this is the reward!
nothate_reward = (logits[:, not_hate_index]).tolist()
print(f'reward (low): {nothate_reward}')

logits [not hate, hate]: [-0.6921170353889465, 0.37227126955986023]
probabilities [not hate, hate]: [0.25647178292274475, 0.7435283064842224]
reward (low): [-0.6921170353889465]


## <b>6.2<span style='color:#78D118'>|</span> Configurar el modelo de recompensa por toxicidad de Pipeline</b>

Utilizamos el pipeline de Hugging Face (HF) para simplificar la inferencia con el modelo de clasificación de toxicidad (RoBERTa).

Notar que aunque el modelo es para "hate speech", usamos la etiqueta "sentiment-analysis" porque: HF usa este nombre genérico para cualquier clasificación binaria de texto.

¿Qué hace el pipeline?

- Tokeniza el texto
- Llama al modelo
- Devuelve los scores (pueden ser logits o probabilidades)


In [ ]:

sentiment_pipe = pipeline("sentiment-analysis",
                          model=toxicity_model_name,
                          framework='pt'
                          ) #device=device
reward_logits_kwargs = {
    "top_k": None, # Devuelve todas las clases (no solo la mejor)
    "function_to_apply": "none", # Devuelve los logits crudos (es decir, sin softmax)
    "batch_size": 16
}

reward_probabilities_kwargs = {
    "top_k": None, # Devuelve todas las clases (no solo la mejor)
    "function_to_apply": "softmax", # Devuelve los logits con la softmax (probabilidades)
    "batch_size": 16
}

print("Reward model output:")
print("For non-toxic text")
print(sentiment_pipe(non_toxic_text, **reward_logits_kwargs))
print(sentiment_pipe(non_toxic_text, **reward_probabilities_kwargs))
print("For toxic text")
print(sentiment_pipe(toxic_text, **reward_logits_kwargs))
print(sentiment_pipe(toxic_text, **reward_probabilities_kwargs))

Device set to use cuda:0


Reward model output:
For non-toxic text
[{'label': 'nothate', 'score': 3.1141021251678467}, {'label': 'hate', 'score': -2.4896185398101807}]
[{'label': 'nothate', 'score': 0.9963293671607971}, {'label': 'hate', 'score': 0.0036706076934933662}]
For toxic text
[{'label': 'hate', 'score': 0.3722698390483856}, {'label': 'nothate', 'score': -0.6921153664588928}]
[{'label': 'hate', 'score': 0.7435277104377747}, {'label': 'nothate', 'score': 0.2564723491668701}]


Los resultados son los logits de las clases `nothate` (positiva) y `hate` (negativa). Pero PPO utilizará los logits solo de la clase `nothate` como señal de recompensa positiva utilizada para ayudar a desintoxicar los resultados de LLM. Como lo vimos anteriormente

## <b>6.3<span style='color:#78D118'>|</span> Evaluar la toxicidad</b>

Para evaluar el desempeño del modelo tanto antes como después de los procesos de ajuste y desintoxicación, es esencial establecer la métrica de evaluación de toxicidad. La puntuación de toxicidad se representa como un valor decimal que va de 0 a 1, donde 1 significa el grado más alto de toxicidad.

Con esto podemos evaluar nuestro modelo (hacer un baseline) antes del PPO y después del PPO (ejemplo):

- Antes de PPO: 0.41 (toxicidad promedio)
- Después de PPO: 0.08 (mucho más bajo)
- Conclusión: El modelo aprendió a ser menos tóxico

La metrica puede ser un AVG de todos los scores de nivel de toxicidad.

In [ ]:
toxicity_evaluator = evaluate.load("toxicity", # nombre de la metrica
                                    toxicity_model_name, #el modelo RoBERTa enternado en hate spech
                                    module_type="measurement", #tipo de metrica (produce valores escalares)
                                    toxic_label="hate") #etiqueta que consideras como tóxica

Device set to use cuda:0


Intente calcular la toxicidad para las mismas oraciones que en la sección anterior. No sorprende que los puntajes de toxicidad sean las probabilidades de la clase "odio" devueltas directamente del modelo de recompensa.


In [ ]:
toxicity_score = toxicity_evaluator.compute(predictions=[
    non_toxic_text
])

print("Toxicity score for non-toxic text:")
print(toxicity_score["toxicity"])

toxicity_score = toxicity_evaluator.compute(predictions=[
    toxic_text
])

print("\nToxicity score for toxic text:")
print(toxicity_score["toxicity"])

Toxicity score for non-toxic text:
[0.0036706076934933662]

Toxicity score for toxic text:
[0.7435277104377747]


Este evaluador se puede utilizar de forma eficaz para calcular los niveles de toxicidad de los diálogos.

Para lograrlo, deberá proporcionar varios componentes esenciales, incluido el conjunto de datos de prueba (`dataset["test"]`), el tokenizador utilizado en la sección antes mencionada, el modelo PEFT previamente congelado y el propio evaluador de toxicidad. Para un enfoque simplificado y organizado, se recomienda encapsular estos procedimientos necesarios dentro de una función dedicada denominada `evaluate_toxicity`.

In [ ]:
def evaluate_toxicity(model,
                      toxicity_evaluator,
                      tokenizer,
                      dataset,
                      num_samples):

    """
    Preprocess the dataset and split it into train and test parts.

    Parameters:
    - model (trl model): Model to be evaluated.
    - toxicity_evaluator (evaluate_modules toxicity metrics): Toxicity evaluator.
    - tokenizer (transformers tokenizer): Tokenizer to be used.
    - dataset (dataset): Input dataset for the evaluation.
    - num_samples (int): Maximum number of samples for the evaluation.

    Returns:
    tuple: A tuple containing two numpy.float64 values:
    - mean (numpy.float64): Mean of the samples toxicity.
    - std (numpy.float64): Standard deviation of the samples toxicity.
    """

    max_new_tokens=100

    toxicities = []
    input_texts = []
    for i, sample in tqdm(enumerate(dataset)):
        input_text = sample["query"]

        if i > num_samples:
            break

        input_ids = tokenizer(input_text, return_tensors="pt", padding=True).input_ids

        generation_config = GenerationConfig(max_new_tokens=max_new_tokens,
                                             top_k=0.0,
                                             top_p=1.0,
                                             do_sample=True)

        response_token_ids = model.generate(input_ids=input_ids,
                                            generation_config=generation_config)

        generated_text = tokenizer.decode(response_token_ids[0], skip_special_tokens=True)

        toxicity_score = toxicity_evaluator.compute(predictions=[(input_text + " " + generated_text)])

        toxicities.extend(toxicity_score["toxicity"])

    # Compute mean & std using np.
    mean = np.mean(toxicities)
    std = np.std(toxicities)

    return mean, std

### Baseline

Y ahora realice el cálculo de la toxicidad del modelo antes del ajuste fino/desintoxicación:

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name) #, device_map="auto"

mean_before_detoxification, std_before_detoxification = evaluate_toxicity(model=ref_model,
                                                                          toxicity_evaluator=toxicity_evaluator,
                                                                          tokenizer=tokenizer,
                                                                          dataset=dataset["test"],
                                                                          num_samples=10)

print(f'toxicity [mean, std] before detox: [{mean_before_detoxification}, {std_before_detoxification}]')

8it [00:46,  5.41s/it]You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
11it [01:03,  5.80s/it]

toxicity [mean, std] before detox: [0.044487327062101525, 0.042698084641549484]


## <b>7 <span style='color:#78D118'>|</span>Desintoxicando los resumenes con PPO</b>

Optimice una política de RL en relación con el modelo de recompensa mediante la optimización de políticas proximales (PPO).


Para la inicialización de `PPOTrainer`, necesitará un collator. En este caso, será una función que transforme los diccionarios de una manera particular. Puede definirlo y probarlo:

In [ ]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

test_data = [{"key1": "value1", "key2": "value2", "key3": "value3"}]
print(f'Collator input: {test_data}')
print(f'Collator output: {collator(test_data)}')

Collator input: [{'key1': 'value1', 'key2': 'value2', 'key3': 'value3'}]
Collator output: {'key1': ['value1'], 'key2': ['value2'], 'key3': ['value3']}


Configurar los parámetros esenciales. Cargar el `ppo_model` y el tokenizador correspondiente.

Además, cargar una versión estática del modelo, denominada `ref_model`.

El propósito de tener dos modelos es doble: el primer modelo, `ppo_model`, se somete a optimización, mientras que el segundo modelo, `ref_model`, funciona como un punto de referencia para calcular la divergencia KL a partir del estado inicial.

Esto sirve como una señal de recompensa adicional en el proceso de entrenamiento PPO, lo que garantiza que el modelo optimizado no se aleje demasiado del modelo de lenguaje (LLM) original.

In [ ]:
learning_rate=1.41e-5
max_ppo_epochs=1
mini_batch_size=4
batch_size=16

config = PPOConfig(
    model_name=model_name,
    learning_rate=learning_rate,
    ppo_epochs=max_ppo_epochs,
    mini_batch_size=mini_batch_size,
    batch_size=batch_size
)

ppo_trainer = PPOTrainer(config=config,
                         model=ppo_model,
                         ref_model=ref_model,
                         tokenizer=tokenizer,
                         dataset=dataset["train"],
                         data_collator=collator)

/usr/local/lib/python3.11/dist-packages/trl/trainer/ppo_config.py:207: FutureWarning: `PPOConfig` is deprecated and will be removed in the future. Please use `PPOv2Config` with `PPOv2Trainer` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/ppo_trainer.py:193: FutureWarning: `PPOTrainer` is deprecated and will be removed in trl v0.12. Please use `PPOv2Trainer` instead.
  warnings.warn(


## <b>7.2 <span style='color:#78D118'>|</span> Entrenando el modelo</b>

El ciclo de ajuste fino comprende los siguientes pasos clave:

1. Recuperar las respuestas de la consulta del modelo de lenguaje de políticas (modelo PEFT).
2. Determinar los sentimientos asociados con las consultas y respuestas utilizando el modelo de discurso de odio RoBERTa.
3. Optimizar la política utilizando la optimización de políticas proximales (PPO) con el triplete de entradas, que incluye la consulta, la respuesta y la recompensa asociada.

Puede confirmar que la operación se está ejecutando correctamente monitoreando las siguientes métricas:

- `objective/kl`: Minimización de la divergencia de Kullback-Leibler (KL).
- `ppo/returns/mean`: Maximización de los retornos medios.
- `ppo/policy/advantages_mean`: Maximización de las ventajas medias.

Estas métricas sirven como indicadores del progreso del proceso de capacitación y el logro de objetivos específicos dentro del ciclo de ajuste fino.

In [ ]:
output_min_length = 100
output_max_length = 400
output_length_sampler = LengthSampler(output_min_length, output_max_length)

generation_kwargs = {
    "min_length": 5,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True
}

reward_kwargs = {
    "top_k": None, # Return all scores.
    "function_to_apply": "none", # You want the raw logits without softmax.
    "batch_size": 16
}

max_ppo_steps = 10

for step, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    # Break when you reach max_steps.
    if step >= max_ppo_steps:
        break

    prompt_tensors = batch["input_ids"]

    # Get response from FLAN-T5/PEFT LLM.
    summary_tensors = []

    for prompt_tensor in prompt_tensors:
        max_new_tokens = output_length_sampler()

        generation_kwargs["max_new_tokens"] = max_new_tokens
        summary = ppo_trainer.generate(prompt_tensor, **generation_kwargs)

        summary_tensors.append(summary.squeeze()[-max_new_tokens:])

    # This needs to be called "response".
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in summary_tensors]

    # Compute reward outputs.
    query_response_pairs = [q + r for q, r in zip(batch["query"], batch["response"])]
    rewards = sentiment_pipe(query_response_pairs, **reward_kwargs)

    # You use the `nothate` item because this is the score for the positive `nothate` class.
    reward_tensors = [torch.tensor(reward[not_hate_index]["score"]) for reward in rewards]

    # Run PPO step.
    stats = ppo_trainer.step(prompt_tensors, summary_tensors, reward_tensors)
    ppo_trainer.log_stats(stats, batch, reward_tensors)

    print(f'objective/kl: {stats["objective/kl"]}')
    print(f'ppo/returns/mean: {stats["ppo/returns/mean"]}')
    print(f'ppo/policy/advantages_mean: {stats["ppo/policy/advantages_mean"]}')
    print('-'.join('' for x in range(100)))

0it [00:00, ?it/s]Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
1it [00:38, 38.47s/it]

objective/kl: 32.16742706298828
ppo/returns/mean: -0.49684637784957886
ppo/policy/advantages_mean: 0.011742033064365387
---------------------------------------------------------------------------------------------------


2it [01:09, 34.22s/it]

objective/kl: 27.3293514251709
ppo/returns/mean: -0.3885064721107483
ppo/policy/advantages_mean: 0.016968464478850365
---------------------------------------------------------------------------------------------------


3it [01:39, 32.27s/it]

...

objective/kl: 22.245620727539062
ppo/returns/mean: -0.13906608521938324
ppo/policy/advantages_mean: 0.01722560077905655
---------------------------------------------------------------------------------------------------


9it [04:40, 29.84s/it]

objective/kl: 23.090007781982422
ppo/returns/mean: -0.19153395295143127
ppo/policy/advantages_mean: 0.026414288207888603
---------------------------------------------------------------------------------------------------


10it [05:06, 30.67s/it]

objective/kl: 21.703269958496094
ppo/returns/mean: 0.018114890903234482
ppo/policy/advantages_mean: 0.01930168643593788
---------------------------------------------------------------------------------------------------


## <b>7.3 <span style='color:#78D118'>|</span>Evaluar el modelo cuantitativamente</b>

Recuperar el modelo PPO/PEFT del punto de control del disco guardado y emplee la división del conjunto de datos de prueba para evaluar la puntuación de toxicidad del modelo ajustado por RL.


In [ ]:
device = 'cpu'
ppo_model = ppo_model.to(device)
ref_model = ref_model.to(device)
#toxicity_evaluator = toxicity_evaluator.to(device)

mean_after_detoxification, std_after_detoxification = evaluate_toxicity(model=ppo_model,
                                                                        toxicity_evaluator=toxicity_evaluator,
                                                                        tokenizer=tokenizer,
                                                                        dataset=dataset["test"],
                                                                        num_samples=10)
print(f'toxicity [mean, std] after detox: [{mean_after_detoxification}, {std_after_detoxification}]')

11it [00:57,  5.23s/it]

toxicity [mean, std] after detox: [0.03398863473822447, 0.040733230166042515]


Comparar los puntajes de toxicidad del modelo de referencia (antes de la desintoxicación) y el modelo ajustado (después de la desintoxicación).

In [ ]:
mean_improvement = (mean_before_detoxification - mean_after_detoxification) / mean_before_detoxification
std_improvement = (std_before_detoxification - std_after_detoxification) / std_before_detoxification

print(f'Percentage improvement of toxicity score after detoxification:')
print(f'mean: {mean_improvement*100:.2f}%')
print(f'std: {std_improvement*100:.2f}%')

Percentage improvement of toxicity score after detoxification:
mean: 23.60%
std: 4.60%


## <b>7.4 <span style='color:#78D118'>|</span>Evaluar el modelo cualitativamente</b>

Explorar ejemplos de muestra del conjunto de datos de prueba, lo que permite una comparación entre el `ref_model` inicial y el `ppo_model` perfeccionado/desintoxicado utilizando el evaluador de toxicidad.

In [ ]:
batch_size = 20
compare_results = {}

df_batch = dataset["test"][0:batch_size]

compare_results["query"] = df_batch["query"]
prompt_tensors = df_batch["input_ids"]

summary_tensors_ref = []
summary_tensors = []

# Get response from ppo and base model.
for i in tqdm(range(batch_size)):
    gen_len = output_length_sampler()
    generation_kwargs["max_new_tokens"] = gen_len

    summary = ref_model.generate(
        input_ids=torch.as_tensor(prompt_tensors[i]).unsqueeze(dim=0),
        **generation_kwargs
    ).squeeze()[-gen_len:]
    summary_tensors_ref.append(summary)

    summary = ppo_model.generate(
        input_ids=torch.as_tensor(prompt_tensors[i]).unsqueeze(dim=0),
        **generation_kwargs
    ).squeeze()[-gen_len:]
    summary_tensors.append(summary)

# Decode responses.
compare_results["response_before"] = [tokenizer.decode(summary_tensors_ref[i]) for i in range(batch_size)]
compare_results["response_after"] = [tokenizer.decode(summary_tensors[i]) for i in range(batch_size)]

# Sentiment analysis of query/response pairs before/after.
texts_before = [d + s for d, s in zip(compare_results["query"], compare_results["response_before"])]
rewards_before = sentiment_pipe(texts_before, **reward_kwargs)
compare_results["reward_before"] = [reward[not_hate_index]["score"] for reward in rewards_before]

texts_after = [d + s for d, s in zip(compare_results["query"], compare_results["response_after"])]
rewards_after = sentiment_pipe(texts_after, **reward_kwargs)
compare_results["reward_after"] = [reward[not_hate_index]["score"] for reward in rewards_after]

pd.set_option('display.max_colwidth', 500)
df_compare_results = pd.DataFrame(compare_results)
df_compare_results["reward_diff"] = df_compare_results['reward_after'] - df_compare_results['reward_before']
df_compare_results_sorted = df_compare_results.sort_values(by=['reward_diff'], ascending=False).reset_index(drop=True)
df_compare_results_sorted

100%|██████████| 20/20 [03:17<00:00,  9.88s/it]


,query,response_before,response_after,reward_before,reward_after,reward_diff
0,"Summarize the following conversation. #Person1#: Judy, what is everybody talking about? #Person2#: Haven't you heard? Richard was fired by our manager. #Person1#: You're kidding. It can't be true. #Person2#: Believe it or not. Everybody is talking about it in the company. #Person1#: Really? I'm surprised. #Person2#: Me too. Summary: </s>","<pad> Judy and Judy are surprised to hear about Richard's electric shock, but Judy isn't surprised.</s>",<pad> Judy praises Richard's story to be had and has asked for her opinion about it in the company.</s>,1.593666,2.010429,0.416763
1,"Summarize the following conversation. #Person1#: Excuse me, could you tell me how to get to the Cross Bakery building? #Person2#: The Cross Bakery building? Oh sure. You're actually walking in the opposite direction. #Person1#: Oh, you're kidding! I thought I was heading east. #Person2#: No, east is the other direction. To get to the Bakery, you need to turn around and go three blocks to Broadway. When you get to the intersection of Broadway and Elm, you hang a left. Go straight down that st...",<pad> #Person2# explains to #Person1# what to do if you're going east to cross the river for cross bakery. #Person1# sold #Person1# the Cross Bakery building and #Person2# would show her the way through the streets.</s>,<pad> #Person1# doesn't know how to get to the Cross Bakery building. #Person2# tells #Person1# the instructions that can go east and stop the exit arrow at the intersection of Broadway and Elm. #Person2# offers to show #Person1# the way to the bakery.</s>,2.721609,3.005683,0.284074
2,"Summarize the following conversation. #Person1#: How much are you asking for this? #Person2#: I'm offering them to you at 150 yuan a piece. Is that all right? #Person1#: Is tax already included in their price? #Person2#: Yes. Our price can't be matched. #Person1#: Would you consider a volume discount? #Person2#: If you buy 1, 000 or more, you'll get a 10 % discount. #Person1#: I'll accept your offer. Summary: </s>","<pad> #Person2# offers them to #Person1# for 150 yuan a piece for any amount, if #Person1# buys more, #Person2# provides a 10 % discount.</s>","<pad> #Person1# fives 150 yuan for a piece, #Person2# requests a 10 % discount and comes with an original head.</s>",2.396964,2.670752,0.273788
3,"Summarize the following conversation. #Person1#: Hello? #Person2#: Hello? #Person1#: Can I speak to Li Hong, please? #Person2#: Speaking. #Person1#: Hi, Li Hong. This is Alice. #Person2#: Hi, Alice. How are you? #Person1#: Not bad. Li Hong, I am sorry that I can't go to see Mrs. Brown with you tomorrow morning. My mother is ill. I must take care of her. #Person2#: I'm sorry to hear that. You'd better stay at home. After all, we can visit Mrs. Brown later #Person1#: OK. Bye - bye. #Person2#: ...",<pad> Alice can't visit Mrs. Brown's tomorrow morning because she's sick. Li Hong invites her for a visit later.</s>,"<pad> Li Hong is sorry to not be able to see Mrs. Brown tomorrow, but Ali will visit her later.</s>",2.040812,2.220668,0.179855
4,"Summarize the following conversation. #Person1#: What can I do for you, madam? #Person2#: I'd like to buy a toy car for my son. #Person1#: How about this one? #Person2#: It looks nice. How much is it? #Person1#: They're three hundred dollars. #Person2#: Oh, I'm afraid it's too expensive. Can you show me something cheaper? #Person1#: OK, This one is one hundred and twenty. It's the cheapest here. #Person2#: OK, I'll take it. Here's the money. #Person1#: Thank you very much. Summary: </s>",<pad> #Person2# is going to buy toy car for #Person2#'s son for $340. #Person1# decides to change an car to one hundred and twenty for #Person2#. But #Person1# advises her to take the one.</s>,<pad> #Person1# asks #Person2# to buy a toy car. They find one city cheaper than the one #Person2# wants.</s>,1.215319,1.385422,0.170103
5,"Summarize the following conversation. #Perso

# Siguientes pasos

Implementar GRPO en este reto dado que es posible por tener grupos de textos inherentes como los topicos. Se puede ser de manera directa tal como se expresa en el siguiente recuadro

Conversión de PPO a GRPO — Tabla Conceptual

| Paso | Componente             | PPO Clásico                                                   | GRPO (Grouped PPO)                                                                 |
|------|------------------------|----------------------------------------------------------------|-------------------------------------------------------------------------------------|
| 1    | *Definición de grupos* | No se usa agrupamiento                                         | Definir group_id para cada muestra (por tema, toxicidad, longitud, etc.)         |
| 2    | *Estructura del dataset* | Prompt + recompensa                                            | Prompt + recompensa + group_id                                                   |
| 3    | *Collator*             | Tokeniza y genera input_ids, attention_mask                | Tokeniza y conserva o agrupa por group_id                                        |
| 4    | *Generación de respuestas* | Se generan con model.generate() para todo el batch          | Se generan igual, pero se pueden organizar por grupo                               |
| 5    | *Cálculo de recompensas* | Se aplica el modelo de recompensa al batch completo            | Se aplica el reward model por grupo (misma lógica, pero evaluado por subgrupo)     |
| 6    | *Estimación de valores* | Una sola cabeza de valor predice V(s_t)                      | Misma cabeza, pero valores y ventajas se calculan por grupo                        |
| 7    | *Cálculo de ventajas*   | $$A_t = r_t - V(s_t)$$ para todo el batch                   | $$A_t^g = r_t^g - V(s_t^g)$$ por grupo                                           |
| 8    | *Pérdida PPO (clipping)* | Se calcula una sola pérdida                                   | Se calcula una pérdida por grupo y luego se combinan (media ponderada o suma)      |
| 9    | *Actualización del modelo* | Una actualización global                                      | Misma política global, pero influenciada por la dinámica de cada grupo             |
| 10   | *Monitoreo y métricas*   | Promedio general de reward, ventaja y KL                      | Métricas separadas por grupo para análisis más fino         